# Limpieza de datos
 
La limpieza de datos es uno de los pasos más importantes en un proyecto de análisis de datos ya que si los datos están "sucios", el análisis también lo será.
 
Para evaluar la validez de los datos de nuestro dataset, es importante analizar y corroborar si los datos cumplen o se ajustan a la reglas o restricciones propias del dato:
* **De tipo de dato**: los valores en una columna en particular deben ser de un tipo de datos.
* **De rango**: generalmente, los números o fechas deben estar dentro de un cierto rango.
* **Obligatorias**: determinadas columnas no pueden estar vacías.
* **Únicas**: un campo, debe ser único en un conjunto de datos.
* **De pertenencia al conjunto**: los valores de una columna provienen de un conjunto de valores discretos. Por ejemplo, el sexo biológico de una persona en general se marca como masculino o femenino.
* **Patrones de expresión regular**: campos de texto que deben seguir un patrón determinado. (Email)
* **Validación de campo cruzado**: deben cumplirse determinadas condiciones que abarcan varios campos. Por ejemplo, la fecha de alta de un paciente del hospital no puede ser anterior a la fecha de admisión.
 

Vamos a hacer una revisión de las columnas para ver si hay datos erroneos. También vamos a revisar si hay datos nulos (registros vacíos). Vamos a resolver estos problemas utilizando métodos de la librería Pandas.

In [ ]:
#importamos las librerias que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns

import numpy as np

In [ ]:
data = pd.read_csv("https://cdn.buenosaires.gob.ar/datosabiertos/datasets/arbolado-publico-lineal/arbolado-publico-lineal-2017-2018.csv")
data.head()

### Errores en la carga de los datos

En muy comun al utilizar datos de la vida real que haya errores en la carga y es necesario realizar una limpieza para normalizar los datos y asegurarse que todo este cargado de la misma manera. Es necesario realizar esto previo a hacer visualizaciones para poder hacer un análisis de datos de mejor calidad.

En este caso análizaremos las columnas *estado_plantera*, *ubicación_planera* y *nivel_planteara* para realizar una limpieza con distintas técnicas de la librearía Pandas.


#### Columna *ubicación_plantera*


In [ ]:
data["ubicacion_plantera"].unique()

En este caso vemos varios problemas: datos que son leídos como diferentes pero en realidad son iguales por diferencia entre mínisculas y mayúsculas o por espacios de más (ejemplo: "Regular "), también se ven campos que están mal escritos como "Ochava" con "Ochva" y "Och" y datos que hay solamente una "o" que puede ser "Ochava" u "Ocupada". Como no lo sabemos en este caso vamos a dejar el dato nulo.

Para poner dato nulo vamos a utilizar un método de la librería Numpy = np.NaN (Not a Number)

Vamos a usar el método método [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) 

In [ ]:
data.replace("Regular ", "Regular", inplace=True)
data.replace("regular", "Regular",inplace=True)
data.replace("o" , np.NaN,inplace=True)
data.replace("O", np.NaN,inplace=True)
data.replace("Och", "Ochava",inplace=True)
data.replace("Ochva", "Ochava",inplace=True)
data.replace("ochava" , "Ochava",inplace=True)
data.replace("Fuera Línea,Ochava", "Ochava/Furea Linea",inplace=True)
data.replace("Fuera de Línea, Ochava", "Ochava/Fuera Linea",inplace=True)
data.replace("Ochava/Fuera Línea", "Ochava/Fuera Linea",inplace=True)
data.replace("Fuera Línea/Ochava", "Ochava/Fuera Linea",inplace=True)
data.replace("Ochava/Furea Linea", "Ochava/Fuera Linea",inplace=True)
              

In [ ]:
data["ubicacion_plantera"].unique()

In [ ]:
data["ubicacion_plantera"].value_counts()

#### Columna *nivel_plantera*


In [ ]:
data["nivel_plantera"].unique()

In [ ]:
data["nivel_plantera"].value_counts()

También notamos que varios valores están escrito de distintas maneras, vamos a utilizar nuevamente el método [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) pero en este caso vamos a crear un diccionario para hacer el reemplazo

In [ ]:
dicc_reemplazar= {"Bajo nivel":"Bajo Nivel",
                  "EL": "Elevadas",
                  "BN": "Bajo Nivel",
                  "Bajo Bivel" : "Bajo Nivel",
                  "Elevado" : "Elevadas",
                  "Eleveda": "Elevadas",
                  "elevada":"Elevadas",
                  "Elevada":"Elevadas",
                  "Baja Nivel": "Bajo Nivel",
                  "ELEVADA": "Elevadas",
                  "el":"Elevadas",
                  "bajo nivel": "Bajo Nivel",
                  "Bajo  nivel": "Bajo Nivel",
                   "obs: no tiene plantera definida": np.NaN
                  }

In [ ]:
data.replace(dicc_reemplazar, inplace=True)

In [ ]:
data["nivel_plantera"].unique()

In [ ]:
# También se puede colocar dentro del reemplace una lista dentro de un diccionario


data.replace(["A nivel","AN",  "A  Nivel","a Nivel", "A nivel ", "A Nivel", "an", "An"], "A Nivel", inplace=True)

In [ ]:
data["nivel_plantera"].unique()

#### Columna  *estado_plantera*

In [ ]:
data["estado_plantera"].unique()

In [ ]:
data["estado_plantera"].value_counts()

Podemos analizar que "Sobreocupada", "sobreocupada" y "SobreOcupada" es el mismo valor, solo que está escrito de distintas maneras ya que python (y la mayoría de los lenguajes) es sensible a las mayusculas y minusculas. Para resolver este tema vamos a utilizar el método [.Lower](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html) de Python y dejar todos los campos en minúsucula

In [ ]:
data["estado_plantera"] = data["estado_plantera"].str.lower()

# Observar que en este caso reemplazamos la columna, podrìamos crear una nueva con otro nombre y luego eliminar la anterior

In [ ]:
data["estado_plantera"].unique()

Si observamos con antención hay un espacio en la segunda "ocupada", por eso esta identificada como distinta, en este caso vamos a utilizar dos métodos de pandas: *apply* y *lambda*

- [apply](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.apply.html): puede realizarse sobre un DataFrame y opera sobre filas o columnas completas o sobre una Series (una sola columna) y opera sobre cada uno de los elementos.

- **funciones lambda** también se denominan funciones anónimas. Una función anónima es una función definida sin un nombre. Como sabemos, para definir una función normal en Python, usamos la palabra clave *def*, pero en el caso de funciones anónimas, usamos la palabra clave *lambda* para definirlas.

La sintaxis es 

    lambda argumento: expresión

Dentro de la expresión se realizará la función que se desea realizar.

En este caso definiremos la función lambda para realizar un condicional donde reemplace los valores "ocuapada " por un "ocupada" y sino que deje el valor.

In [ ]:
# Se define una nueva columna "estado_plantera" con "apply" se corre la función "lambda" por cada uno de los valores de las celdas de nuestra Series
# x es el argumento que en este caso se refiere a cada valor de la columna

data["estado_plantera"] = data["estado_plantera"].apply(lambda x: "ocupada" if x=="ocupada " else x)

En el mismo sentido podemos considerar que "cantero ocupado" y "ocupada" son lo mismo, está es una decisión del/la analista de datos teniendo conociemiento sobre el tema que estamos trabajando. 

Vamos a utilizar nuevamente el método [replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html)

In [ ]:
data.replace("cantero ocupado", "ocupada", inplace=True)

In [ ]:
data["estado_plantera"].unique()

#### Identificación de datos nulos

Ahora vamos a limpiar los datos nulos que encontramos en nuestro dataset. 
Para observar si hay datos nulos se puede utilizar el método *isnull()* o *isna()* el cual devovlerá **True** en el caso que haya un dato nulo y **False** en el caso que no lo haya.
Esto puede ser utilizado como una máscara para filtrar el dataset y ver los registros nulos.

Para saber la cantidad de datos nulos es posible utilizar *sum()* luego de la identificación de los nulos sabiendo que **True** cuenta como 1 y **False** cuenta como 0, de esta manera sumara 1 por cada dato nulo. 

In [ ]:
data.isnull()

In [ ]:
data.isna()

In [ ]:
data.isnull().sum()

Podemos observar que hay varias variables quue tienen datos nulos, frente a eso se pueden realizar distintas estrategias: **imputar**, **marcar** o **eliminar**, eso dependerá de la importancia de la columna, de la cantida de datos nulos y es decisión del/la Analista de datos. 

#### Columna *manzana*

En este caso vamos a analizar qué columnas no son importantes para nuestro análisis por ejemplo la columna *manzana*, se interpreta que tiene muchos nulos porque no en todas las direcciones se indica este dato, pero vemos que de 370.180, 146.040 son nulos, es decir casi el 40% es nulo.

En este caso vamos a eliminar la columna con el método [drop](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)

In [ ]:
data.drop(columns="manzana", inplace=True)

In [ ]:
data.columns

#### Columna *calle_nombre*

Vamos a ver los registros nulos de de *calle_nombre*, para eso vamos a hacer una máscara y filtrar el dataset para verlo mejor

In [ ]:
mask_nulo_calle = data["calle_nombre"].isnull()
mask_nulo_calle

In [ ]:
data.loc[mask_nulo_calle]

En este caso observamos que los registros que tienen nulo el *calle_nombre* tienen varios datos faltantes de muchas columnas, no tenemos ningún dato de ubicación por lo que en este caso tal vez vamos a eliminar los registros que tienen este dato nulo

En este caso lo que realizaremos es **eliminar** las filas que tienen muchos datos nulos utilizando el método [dropna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html). Este método permite eliminar filas o columnas y se pueden definir distintos criterios. En este caso eliminaremos los registos que tienen nulo esta columna.

In [ ]:
data.dropna(subset=["calle_nombre"], inplace=True)

In [ ]:
# Revisamos la operación

data.shape

In [ ]:
data.isnull().sum()

#### Columna *nivel_plantera*

En este caso vemos que hay 1324 nulos que representan menos del 1% de los datos

In [ ]:
data.loc[data["nivel_plantera"].isnull()]

Podemos observar que es solamente uno o dos datos y no se observa ninguna otra particularidad ya que el resto de los valores parecen correctos.

En este caso lo que realizaremos es **imputar** de manera aleatoria el dato faltante. Al ser una variable categórica utilizaremos la **moda**, es decir, el valor que más veces se repite. Para esto volveremos a observar como estás distribuidos los valores que puede tomar la columna y luego definiremos con *.loc* el dato faltante y lo completaremos.

En este caso lo realizaremos de manera aleatoria utilizando como criterio la **moda**, también podría imputarse los datos faltantes en base a los otros datos de las columnas que si tenemos. 

In [ ]:
data["nivel_plantera"].value_counts()

In [ ]:
# Como la moda es "A Nivel" (el valor que más se repite) imputaremos con ese valor (que además corresponde con la humedad)

# Seleccionaremos el dato que queremos imputar

data.loc[data["nivel_plantera"].isnull()] = "A Nivel"

In [ ]:
# Revisamos la operación

data.loc[data["nivel_plantera"].isnull()]

In [ ]:
# Comprobamos que este correctamente imputado y que no haya más datos nulos en lluvia

print(data["nivel_plantera"].value_counts())
data.isnull().sum()

#### Columna *ubicacion_plantera*

Ahora realizaremos el mismo procedimiento en el caso de los datos nulos en la variable *ubicacion_plantera*, utilizaremos *isnull()* para crear una máscara y luego filtraremos con *.loc.* para observar los datos

In [ ]:
data.loc[data["ubicacion_plantera"].isnull()]

En este caso nuevamente lo que realizaremos es **imputar** de manera aleatoria el dato faltante. Para esto volveremos a observar los datos que puede tomar esta columna y luego definiremos con *.loc* el dato faltante y lo completaremos. Esta imputación será creando una lista aleatoria de valores para completar los datos faltantes. 

Para crear esta lista utilizaremos la libreria *random* cuyo método *choices* permite la generación de una lista aleatoria de los 4 valores que más se repiten. Para ver más de esta librería y practicar puede consultaste [w3schools](https://www.w3schools.com/python/ref_random_choice.asp)

Aquí también lo realizaremos de manera aleatoria, pero podría imputarse los datos faltantes en base a los otros datos de las columnas que si tenemos.

In [ ]:
# Primero confirmaremos los valores que puede tomar la columna "descripcion"

data["ubicacion_plantera"].value_counts()

In [ ]:
# Creamos la lista que tome los valores que indiquemos y k es la cantidad de valores que se generaran, en este caso 629

import random

lista_imputar = random.choices(["Regular", "Ochava", "Fuera de linea", "A Nivel"], k=629)  #correr varias veces para ver como se modifica
lista_imputar

In [ ]:
# Volvemos a seleccionar los campos que queremos imputar y definimos los campos con la lista creada en el paso anterior

data.loc[data["ubicacion_plantera"].isnull(), "ubicacion_plantera"]=lista_imputar

In [ ]:
# Comprobamos que este correctamente imputado y que no haya más datos nulos en lluvia

print(data["ubicacion_plantera"].value_counts())

data.isnull().sum()

## Conclusión

En esta notebook hemos revisado disntias formas de resolver los datos erroneos faltantes y se vieron diferentes métodos útiles para realizar estas operaciones. Hay muchos métodos de Pandas que se pueden utilizar como por ejemplo [fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html), en este caso solo hemos visto algunos, es importante resaltar que en todo el proceso es el/la Analista de Datos quien debe tomar decisiones sobre los pasos a seguir.

### Desafío 6: Buscar Datasets

El objetivo es ponerse en grupos y buscar dataset que sean de su interés para poder realizar un análisis. Puede ser de cualquier tema que les interese que puedan contar con Datos. Les dejamos un Documento de Google con sitios donde pueden buscar datasets, es un Documento colaborativo entre todas las comisiones de Datos asì que pueden agregar: [Dataset](https://docs.google.com/document/d/1EnimkrV_a9cu-ZrrcydZnigquQeGSpUsnqu0BA3fUFE/edit?usp=sharing!)